In [1]:
import json

file_path = "../train.json"
with open(file_path, "r") as file:
    data = json.load(file)

In [2]:
all_orgs = list(set([item['filename'].split('/')[0] for item in data]))
len(all_orgs)

133

In [23]:
# WHAT IF I DO NOT USE VECTOR SEARCH, and SEND ALL THE DOCS FOR ONE ORG IN ONE CALL
import tiktoken

results_for_df = []

for org in all_orgs:

    reformatted_data = {}
    for item in data:

        if item['filename'].split('/')[0] == org:
                
            # Define empty list to add the data
            selected_data = []

            # First add the text and tables
            selected_data.append(f"""Pre-Text:{str(item['pre_text'])} \n
                                        Table: {str(item['table'])} \n
                                        Post-Text: {str(item['post_text'])}""")
            
            # Also adding questions for reference
            if item.get('qa'):
                selected_data.append(item['qa']['question'])
            else:
                # If there are multiple questions, just adding the context. So we dont miss any row
                selected_data.append('')

            reformatted_data [item['id']] = selected_data



    # Choose the correct encoding based on the OpenAI model
    encoding = tiktoken.get_encoding("cl100k_base") 

    total_tokens = 0
    num_docs = 0
    for datakey in reformatted_data:
        current_tokens = len(encoding.encode(reformatted_data[datakey][0]))
        num_docs+=1
        total_tokens += current_tokens

    # Store results in a list
    results_for_df.append({
        "ORG": org,
        "TOTAL_TOKENS": total_tokens,
        "TOTAL DOCUMENTS": num_docs,
        "GPT-4o_Price": total_tokens * 2.5 / 1_000_000,
        "GPT-4o-mini_Price": total_tokens * 0.15 / 1_000_000,
        "EXCEEDS_LIMIT": total_tokens > 128_000
    })

    # print(f"Price for Embedding V3 Small -> {total_tokens*0.02/1000000}")
    # print(f"Price for Embedding V3 Large -> {total_tokens*0.13/1000000}")
    print(f"ORG: {org},  TOTAL CONTEXT TOKENS: {total_tokens}")
    if total_tokens>128000:
        print("EXCEEDING:", total_tokens)
    print(f"Price for GPT-4o      ---> {total_tokens*2.5/1000000}")
    print(f"Price for GPT-4o-mini ---> {total_tokens*0.15/1000000}")
    print("")


ORG: EW,  TOTAL CONTEXT TOKENS: 10695
Price for GPT-4o      ---> 0.0267375
Price for GPT-4o-mini ---> 0.00160425

ORG: ADBE,  TOTAL CONTEXT TOKENS: 41773
Price for GPT-4o      ---> 0.1044325
Price for GPT-4o-mini ---> 0.00626595

ORG: CME,  TOTAL CONTEXT TOKENS: 23807
Price for GPT-4o      ---> 0.0595175
Price for GPT-4o-mini ---> 0.0035710499999999997

ORG: CB,  TOTAL CONTEXT TOKENS: 23030
Price for GPT-4o      ---> 0.057575
Price for GPT-4o-mini ---> 0.0034545

ORG: MRO,  TOTAL CONTEXT TOKENS: 76030
Price for GPT-4o      ---> 0.190075
Price for GPT-4o-mini ---> 0.0114045

ORG: SWKS,  TOTAL CONTEXT TOKENS: 9442
Price for GPT-4o      ---> 0.023605
Price for GPT-4o-mini ---> 0.0014163

ORG: ETFC,  TOTAL CONTEXT TOKENS: 9135
Price for GPT-4o      ---> 0.0228375
Price for GPT-4o-mini ---> 0.00137025

ORG: JKHY,  TOTAL CONTEXT TOKENS: 6137
Price for GPT-4o      ---> 0.0153425
Price for GPT-4o-mini ---> 0.0009205499999999999

ORG: UAA,  TOTAL CONTEXT TOKENS: 12794
Price for GPT-4o      --->

In [24]:
import pandas as pd

results_df = pd.DataFrame(results_for_df)
results_df

,ORG,TOTAL_TOKENS,TOTAL DOCUMENTS,GPT-4o_Price,GPT-4o-mini_Price,EXCEEDS_LIMIT
0,EW,10695,15,0.026738,0.001604,False
1,ADBE,41773,36,0.104432,0.006266,False
2,CME,23807,28,0.059518,0.003571,False
3,CB,23030,26,0.057575,0.003455,False
4,MRO,76030,76,0.190075,0.011404,False
...,...,...,...,...,...,...
128,FBHS,2961,3,0.007403,0.000444,False
129,ILMN,11490,14,0.028725,0.001723,False
130,CDNS,11354,18,0.028385,0.001703,False
131,ABMD,13888,18,0.034720,0.002083,False


In [25]:
# Display the DataFrame sorted by GPT-4o_Price in descending order
sorted_by_tokens = results_df.sort_values(by="TOTAL_TOKENS", ascending=False)
sorted_by_tokens

,ORG,TOTAL_TOKENS,TOTAL DOCUMENTS,GPT-4o_Price,GPT-4o-mini_Price,EXCEEDS_LIMIT
71,ETR,148980,198,0.372450,0.022347,True
14,AMT,110506,115,0.276265,0.016576,False
26,UNP,108684,125,0.271710,0.016303,False
41,LMT,106161,100,0.265402,0.015924,False
113,IP,91860,71,0.229650,0.013779,False
...,...,...,...,...,...,...
69,SPGI,1184,1,0.002960,0.000178,False
66,SYY,1113,1,0.002782,0.000167,False
127,TXN,1090,1,0.002725,0.000163,False
17,FTV,1029,1,0.002573,0.000154,False


In [ ]:
etr_tokens_per_doc = 148980/198
etr_tokens_per_doc * 20

15048.484848484848

In [ ]:

# Choose the correct encoding based on the OpenAI model
encoding = tiktoken.get_encoding("cl100k_base") 

total_tokens = 0
for datakey in reformatted_data:
    current_tokens = len(encoding.encode(reformatted_data[datakey][0]))
    
    total_tokens += current_tokens


# print(f"Price for Embedding V3 Small -> {total_tokens*0.02/1000000}")
# print(f"Price for Embedding V3 Large -> {total_tokens*0.13/1000000}")
print(f"ORG: {org},  TOTAL CONTEXT TOKENS: {total_tokens}")
if total_tokens>128000:
    print("EXCEEDING:", total_tokens)
print(f"Price for GPT-4o      ---> {total_tokens*2.5/1000000}")
print(f"Price for GPT-4o-mini ---> {total_tokens*0.15/1000000}")
print("")
